In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r'..\data\replay_data\replay_data_public.DSK.PremierDraft.csv', nrows = 100)


In [3]:
def map_id_to_winrate(value, id_to_wr_mapping):
    """
    Maps a single card ID to its winrate using the provided dictionary.
    
    Parameters:
        value (int): The card ID to map.
        id_to_wr_mapping (dict): A dictionary mapping card IDs to winrates.
    
    Returns:
        float: The winrate of the card ID, or NaN if not found.
    """
    value = int(value)
    return id_to_wr_mapping.get(value, np.nan)

   

def get_card_columns(df, column_prefix, max_cards):
    """
    Identifies columns in the DataFrame corresponding to card slots (e.g., user_hand_1, user_hand_2, ...).
    
    Parameters:
        df (pd.DataFrame): The DataFrame to check for card columns.
        column_prefix (str): The prefix for the column names (e.g., 'user_hand').
        max_cards (int): The maximum number of card slots to check for.
    
    Returns:
        list: A list of column names that exist in the DataFrame matching the prefix and range.
    """
    return [f'{column_prefix}_{i}' for i in range(1, max_cards + 1) if f'{column_prefix}_{i}' in df.columns]

# Example usage
def load_id_to_wr_mapping():
    """
    Loads and prepares a card mapping dictionary (ID → GP WR).
    
    Returns:
        dict: A dictionary mapping card IDs to GP WR.
    """
    cards = pd.read_csv('../data/cards_data/cards.csv')
    ratings = pd.read_csv('../data/cards_data/card_ratings/dks_card_ratings.csv')
    card_mapping = cards.merge(ratings, left_on='name', right_on='Name')[['id', 'GP WR']]
    # convert 'GP WR' to numeric, remove % and move decimal
    card_mapping['GP WR'] = card_mapping['GP WR'].str.rstrip('%').astype('float') / 100.0
    return card_mapping.set_index('id')['GP WR'].to_dict()
def apply_winrate_mapping(df, id_to_wr_mapping, column_prefix, max_cards):
    """
    Applies map_id_to_winrate to the specified card columns in the DataFrame.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the card ID columns.
        id_to_wr_mapping (dict): A dictionary mapping card IDs to GP WR values.
        column_prefix (str): The prefix for the column names (e.g., 'user_hand').
        max_cards (int): The maximum number of card slots to check for.
    
    Returns:
        pd.DataFrame: The updated DataFrame with card IDs replaced by GP WR values.
    """
    card_columns = get_card_columns(df, column_prefix, max_cards)
    
    for col in card_columns:
        df[col] = df[col].apply(lambda x: map_id_to_winrate(x, id_to_wr_mapping) if pd.notnull(x) else np.nan)
    
    return df


In [4]:
def count_player_lands(player_land_value):
    """
    count the number of '|' in the string return that number + 1. If the value is NaN return 0

    """
    player_land_value = str(player_land_value)
    
    if pd.isna(player_land_value):
        return 0

    return player_land_value.count('|') + 1

In [5]:
def explode_cards(card_string, max_cards=20):
    """
    Explodes a delimited card string into a fixed-length list with NaN for missing values.
    
    Parameters:
        card_string (str): The string containing delimited card numbers.
        max_cards (int): The maximum number of card slots to return.
        
    Returns:
        list: A list of length `max_cards` containing card numbers or NaN.
    """
    if not isinstance(card_string, str) or not card_string:
        return [np.nan] * max_cards  # Handle empty or non-string input
    
    cards = card_string.split('|')  # Split the string into a list
    exploded = cards[:max_cards] + [np.nan] * (max_cards - len(cards))  # Pad with NaN if fewer than max_cards
    return exploded

def generate_card_columns(exploded_cards, prefix):
    """
    Generate a dictionary for card columns with appropriate names.
    
    Parameters:
        exploded_cards (list): A list of card values (e.g., from `explode_cards`).
        prefix (str): The prefix for the column names (e.g., 'user_hand').
    
    Returns:
        dict: A dictionary with keys like 'user_hand_1', 'user_hand_2', ..., and corresponding card values.
    """
    return {f'{prefix}_{i+1}': exploded_cards[i] for i in range(len(exploded_cards))}



In [6]:
def transform_row_to_turns(row, id_to_wr_mapping, max_cards=20):
    """
    Transforms a single game row into multiple rows, one for each turn,
    and replaces card IDs with GP WR values.
    
    Parameters:
        row (pd.Series): A single row from the original dataset.
        id_to_wr_mapping (dict): A dictionary mapping card IDs to GP WR values.
        max_cards (int): Maximum number of slots for cards in hand or on the board.
    
    Returns:
        pd.DataFrame: A DataFrame where each row represents a turn with GP WR values.
    """
    turns = []  # Use a list to collect all turn data dictionaries
    
    for turn in range(1, row['num_turns'] + 1):  # Iterate through the turns
        # Create a dictionary to hold features for the current turn
        turn_data = {
            'game_id': row['draft_id'],  # Unique identifier for the game
            'turn': turn,
            'on_play': row.get('on_play', None),
            'won': row.get('won', None),  # Target variable
        }
        
        # Add game state metrics for user and opponent dynamically
        for player in ['user', 'oppo']:
            # Handle cards in hand
            if player == "user":
                cards_in_hand_str = row.get(f'{player}_turn_{turn}_eot_{player}_cards_in_hand', None)
                exploded_cards = explode_cards(cards_in_hand_str, max_cards=max_cards)
                card_columns = generate_card_columns(exploded_cards, f'{player}_hand')
                turn_data.update(card_columns)
            if player == "oppo":
                turn_data[f'{player}_cards_in_hand'] = row.get(f'{player}_turn_{turn}_eot_{player}_cards_in_hand', None)

            # Handle lands
            land_values = row.get(f'{player}_turn_{turn}_eot_{player}_lands_in_play', None)
            turn_data[f'{player}_lands_in_play'] = count_player_lands(land_values)
            
            # Handle creatures
            creatures_in_play_str = row.get(f'{player}_turn_{turn}_eot_{player}_creatures_in_play', None)
            exploded_creatures = explode_cards(creatures_in_play_str, max_cards=max_cards)
            creature_columns = generate_card_columns(exploded_creatures, f'{player}_creatures')
            turn_data.update(creature_columns)
            
            # Handle non-creatures
            non_creatures_in_play_str = row.get(f'{player}_turn_{turn}_eot_{player}_non_creatures_in_play', None)
            exploded_non_creatures = explode_cards(non_creatures_in_play_str, max_cards=max_cards)
            non_creature_columns = generate_card_columns(exploded_non_creatures, f'{player}_non_creatures')
            turn_data.update(non_creature_columns)
            
            # Handle life
            turn_data[f'{player}_life'] = row.get(f'{player}_turn_{turn}_eot_{player}_life', None)
        
        # Append the turn data dictionary to the list
        turns.append(turn_data)

    # Convert the list of dictionaries into a DataFrame
    turn_df = pd.DataFrame(turns)
    
    # Apply ID-to-winrate mapping for relevant columns
    for prefix in ['user_hand', 'oppo_hand', 'user_creatures', 'oppo_creatures', 'user_non_creatures', 'oppo_non_creatures']:
        turn_df = apply_winrate_mapping(turn_df, id_to_wr_mapping, prefix, max_cards=max_cards)
    
    return turn_df


In [7]:
# Example usage
id_to_wr_mapping = load_id_to_wr_mapping()
sample_row = data.iloc[0]
sample_df = transform_row_to_turns(sample_row, id_to_wr_mapping)
sample_df

,game_id,turn,on_play,won,user_hand_1,user_hand_2,user_hand_3,user_hand_4,user_hand_5,user_hand_6,...,oppo_non_creatures_12,oppo_non_creatures_13,oppo_non_creatures_14,oppo_non_creatures_15,oppo_non_creatures_16,oppo_non_creatures_17,oppo_non_creatures_18,oppo_non_creatures_19,oppo_non_creatures_20,oppo_life
0,53401b113a4f425fa26e60edd314dd27,1,True,True,0.555,0.541,0.551,0.558,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
1,53401b113a4f425fa26e60edd314dd27,2,True,True,0.559,0.555,0.551,0.558,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
2,53401b113a4f425fa26e60edd314dd27,3,True,True,NaN,0.559,0.555,0.551,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0
3,53401b113a4f425fa26e60edd314dd27,4,True,True,0.555,0.551,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
4,53401b113a4f425fa26e60edd314dd27,5,True,True,0.555,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [8]:
# run on the whole dataset

def transform_replay_data(data, id_to_wr_mapping, max_cards=20):
    """
    Transforms the replay data into a format suitable for modeling.
    
    Parameters:
        data (pd.DataFrame): The replay data to transform.
        id_to_wr_mapping (dict): A dictionary mapping card IDs to GP WR values.
        max_cards (int): Maximum number of slots for cards in hand or on the board.
    
    Returns:
        pd.DataFrame: The transformed replay data with GP WR values and one row per turn.
    """
    # Initialize an empty list to collect all turn DataFrames
    all_turns = []
    
    # Iterate through each row in the data
    for idx, row in data.iterrows():
        # Transform the row into a DataFrame of turns
        turn_df = transform_row_to_turns(row, id_to_wr_mapping, max_cards=max_cards)
        
        # Append the DataFrame to the list
        all_turns.append(turn_df)
    
    # Concatenate all DataFrames into a single DataFrame
    all_turns_df = pd.concat(all_turns, ignore_index=True)
    return all_turns_df

In [9]:
# run  on the whole dataset

total_data = transform_replay_data(data, id_to_wr_mapping)

In [10]:
total_data.to_csv('../data/replay_data/samples/turn_based_replay_data_transform.csv')